<a href="https://colab.research.google.com/github/m-r-tanha/Deep-Learning-RCNN-Anomaly-detection/blob/master/RCNN02_TF_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, normalize, minmax_scale
#from tensorflow.contrib import rnn
#from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RNN, GRU
import numpy as np
from google.colab import files
import io
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from keras.utils import np_utils
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split

#tbc=TensorBoardColab()

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Wait for 8 seconds...
TensorBoard link:
https://7fb61dce3318.ngrok.io


Saving 1000_Sample3.xlsx to 1000_Sample3.xlsx
User uploaded file "1000_Sample3.xlsx" with length 452161 bytes


In [ ]:
from keras.utils import np_utils
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM

In [ ]:
writer = pd.ExcelWriter('Yexcel.xlsx')
data_path="1000_Sample3.xlsx"
data=pd.read_excel(data_path,sheet_name='Sheet3')
data=data.fillna(0.0)
data.rename(index=data.Cell,inplace=True)
data.drop('Cell',axis=1,inplace=True)
data.Lable_target=data.Lable_target.replace(to_replace=['N','DS','SI','GD','SD','GI'],value=[1,2,3,4,5,6])
X=data.drop('Lable_target',axis=1).values
Y=data.Lable_target
#print(Y)
#Y.describe()


In [ ]:
# creating a noise with the same dimension as the dataset 
mu, sigma = .75, 0.1 
noise = np.random.normal(mu, sigma, [1052,31]) 
noise.shape
#-------------

wn = np.random.randn(len(X),31)
data_wn=X+wn

X=np.append (X,data_wn,axis=0)
Y=np.append(Y,Y,axis=0)
#print(X)

In [ ]:
Normalized_X=scale(X, axis=1)
Mi=Normalized_X.min()   
Normalized_X=(-1*Mi)+Normalized_X
X_train,X_test,Y_train, Y_test = train_test_split(Normalized_X,Y,test_size=.2,random_state=42)

In [ ]:
import tensorflow.compat.v1 as tf
import keras
tf.disable_v2_behavior() 
tf.reset_default_graph()

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

Y_train = np.delete(Y_train , 0, 1)
Y_test = np.delete(Y_test , 0, 1)
# Training Parametersznum_steps = 30
num_steps=500
batch_size = 120
display_step = 1
strides = 1
k = 1

# Network Parameters
num_input = 31  #  data input (img shape: 28*28)
num_hidden = 100
num_classes = 6  #  total classes (0-9 digits)
dropout = 0.7  # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

is_training = tf.placeholder(tf.bool, name='MODE')


# Store layers weight & bias
# The first three convolutional layer
w_c_1 = tf.Variable(tf.random_normal([1, 3, 1, 28]))
w_c_2 = tf.Variable(tf.random_normal([1, 3, 28, 56]))
w_c_3 = tf.Variable(tf.random_normal([1, 3, 56, 112]))
b_c_1 = tf.Variable(tf.zeros([28]))
b_c_2 = tf.Variable(tf.zeros([56]))
b_c_3 = tf.Variable(tf.zeros([112]))

# The second three convolutional layer weights
w_c_4 = tf.Variable(tf.random_normal([1, 3, 112, 224]))
w_c_5 = tf.Variable(tf.random_normal([1, 3, 224, 448]))
w_c_6 = tf.Variable(tf.random_normal([1, 3, 448, 896]))
b_c_4 = tf.Variable(tf.zeros([224]))
b_c_5 = tf.Variable(tf.zeros([448]))
b_c_6 = tf.Variable(tf.zeros([896]))

# Fully connected weight
w_f_1 = tf.Variable(tf.random_normal([1 * 31 * 896, 1792])) # fully connected, 1*3*896 inputs, 2048 outputs
w_f_2 = tf.Variable(tf.random_normal([1792, 896]))
w_f_3 = tf.Variable(tf.random_normal([896, 448]))
b_f_1 = tf.Variable(tf.zeros([1792]))
b_f_2 = tf.Variable(tf.zeros([896]))
b_f_3 = tf.Variable(tf.zeros([448]))

# output layer weight
w_out = tf.Variable(tf.random_normal([448, num_classes]))
b_out = tf.Variable(tf.zeros([num_classes]))

#
# Define model
x = tf.reshape(X, shape=[-1, 1, 31, 1])


# first layer convolution
conv1 = tf.nn.conv2d(x, w_c_1, strides=[1, 1, 1, 1], padding='SAME') + b_c_1
conv1 = tf.compat.v1.layers.batch_normalization(conv1, training=True, center=True, scale=False)
#conv1 = BatchNormalization(conv1, center=True, scale=False)
conv1 = tf.nn.tanh(conv1)


# second layer convolution
conv2 = tf.nn.conv2d(conv1, w_c_2, strides=[1, strides, strides, 1], padding='SAME') + b_c_2
conv2 = tf.compat.v1.layers.batch_normalization(conv2, training=True, center=True, scale=False)
conv2 = tf.nn.tanh(conv2)
conv2 = tf.nn.dropout(conv2, dropout)

# third layer convolution
conv3 = tf.nn.conv2d(conv2, w_c_3, strides=[1, strides, strides, 1], padding='SAME') + b_c_3
conv3 = tf.compat.v1.layers.batch_normalization(conv3, training=True, center=True, scale=False)
conv3 = tf.nn.tanh(conv3)

# first Max Pooling (down-sampling)
pool_1 = tf.nn.max_pool(conv3, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# fourth layer convolution
conv4 = tf.nn.conv2d(pool_1, w_c_4, strides=[1, strides, strides, 1], padding='SAME') + b_c_4
conv4 = tf.compat.v1.layers.batch_normalization(conv4, training=True, center=True, scale=False)
conv4 = tf.nn.tanh(conv4)
conv4 = tf.nn.dropout(conv4, dropout)

# fifth layer convolution
conv5 = tf.nn.conv2d(conv4, w_c_5, strides=[1, strides, strides, 1], padding='SAME') + b_c_5
conv5 = tf.compat.v1.layers.batch_normalization(conv5, training=True, center=True, scale=False)
conv5 = tf.nn.tanh(conv5)

# sixth layer convolution
conv6 = tf.nn.conv2d(conv5, w_c_6, strides=[1, strides, strides, 1], padding='SAME') + b_c_6
conv6 = tf.compat.v1.layers.batch_normalization(conv6, training=True, center=True, scale=False)
conv6 = tf.nn.tanh(conv6)
conv6 = tf.nn.dropout(conv6, dropout)

# second Max Pooling (down-sampling)
pool_2 = tf.nn.max_pool(conv6, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

pool_2 = tf.reshape(pool_2, [-1,31,896])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden,num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def RNN(x, weights, biases):
  
    lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_hidden,reuse=tf.AUTO_REUSE)
    outputs, states = tf.compat.v1.nn.dynamic_rnn(lstm_cell, x , dtype=tf.float32)
    return tf.matmul(outputs[:,-1], weights['out']) + biases['out']

logits = RNN(pool_2, weights, biases)
prediction = tf.nn.softmax(logits)

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$4
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

optimizer = tf.compat.v1.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Please don't change these.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#writer.close()
A=pd.DataFrame({'loss':[], 'Acc':[]})

# Start training
with tf.Session(config=config) as sess:
    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps + 1):

        sess.run(train_op, feed_dict={X: X_train, Y: Y_train, keep_prob: 0.7})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc= sess.run([loss_op, accuracy], feed_dict={X: X_train,
                                                                 Y: Y_train,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) )
            df=pd.DataFrame({'loss':[loss], 'Acc':[acc]})
            A=A.append(df) 
    saver=tf.train.Saver()
    save_path=saver.save(sess,"./TF_ModelwBN_for_Paper4.ckpt")
    #tbc.save_value("graph_name", "line_name", num_steps, loss)
    #tbc.flush_line(line_name)
    #tbc.close()
    print("Optimization Finished!")    
    print("Testing Accuracy:", \
          sess.run(accuracy, feed_dict={X: X_test , Y: Y_test}))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/legacy_tf_layers/normalization.py:308: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  '`tf.layers.batch_normalization` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and wil

Step 1, Minibatch Loss= 2.6017, Training Accuracy= 0.466
Step 2, Minibatch Loss= 1.8667, Training Accuracy= 0.462
Step 3, Minibatch Loss= 1.6949, Training Accuracy= 0.495
Step 4, Minibatch Loss= 1.6860, Training Accuracy= 0.504
Step 5, Minibatch Loss= 1.4150, Training Accuracy= 0.529
Step 6, Minibatch Loss= 1.4023, Training Accuracy= 0.533
Step 7, Minibatch Loss= 1.3587, Training Accuracy= 0.541
Step 8, Minibatch Loss= 1.3395, Training Accuracy= 0.541
Step 9, Minibatch Loss= 1.2171, Training Accuracy= 0.562
Step 10, Minibatch Loss= 1.1864, Training Accuracy= 0.562
Step 11, Minibatch Loss= 1.1346, Training Accuracy= 0.570
Step 12, Minibatch Loss= 1.1247, Training Accuracy= 0.557
Step 13, Minibatch Loss= 1.1213, Training Accuracy= 0.563
Step 14, Minibatch Loss= 1.0694, Training Accuracy= 0.565
Step 15, Minibatch Loss= 1.0607, Training Accuracy= 0.556
Step 16, Minibatch Loss= 0.9972, Training Accuracy= 0.589
Step 17, Minibatch Loss= 1.0058, Training Accuracy= 0.578
Step 18, Minibatch Loss

In [ ]:
outputs

NameError: ignored

In [ ]:
print(tf.__version__)

2.4.1
